# Distance Model Integration
BS"D

In this notebook, I will explore how to integrate the filtering model and the similar ingredients model to create the distance model.

The API will accept a list of ingredients (a recipe) and a set of dietary restrictions. It will return a new recipe that meets the dietary restrictions and is similar to the original recipe.